In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pandas datasets langchain langchain-openai langchain-core tqdm openai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.

## LLM 기반 타겟 모델 최적화 데이터 전처리

###이 노트북은 금융 Q&A 데이터를 로드하여 대규모 언어 모델(LLM, 여기서는 GPT-4o)을 사용해 전처리합니다.
###최종적으로 사용할 모델의 유형(`RAG` 또는 `Fine-tuning`)에 따라 최적화된 전처리 작업을 수행합니다.

## **주요 단계:**
#### 1. 필요한 라이브러리 임포트
#### 2. 실행 설정 및 OpenAI API 키 구성
#### 3. LLM 클라이언트 초기화
#### 4. 데이터 로딩 및 전처리 함수 정의
#### 5. 데이터 로드
#### 6. 타겟 모델 유형에 따른 LLM 전처리 실행
#### 7. 결과 확인 및 저장

In [6]:
# === 라이브러리 임포트 ===
import os
import pandas as pd
import json
import time
import traceback
from typing import Optional, Tuple, List, Dict, Any

# Google Colab 환경 확인
try:
    from google.colab import userdata, drive
    IS_COLAB = True
    print("Google Colab 환경 감지됨.")
except ImportError:
    IS_COLAB = False
    print("로컬 또는 다른 환경 감지됨.")

# LangChain 및 관련 라이브러리
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.prompts import PromptTemplate
from datasets import load_dataset
from tqdm.auto import tqdm # 진행률 표시줄 라이브러리

# Pandas 진행률 표시줄 활성화
tqdm.pandas()

Google Colab 환경 감지됨.


In [7]:
# 1. 설정 및 API 키 구성
#
# **중요:** 아래 셀에서 `YOUR_OPENAI_API_KEY` 부분을 실제 OpenAI API 키로 바꿔주세요.
# Google Colab을 사용하는 경우, Colab의 'Secrets' 기능에 `OPENAI_API_KEY`라는 이름으로 키를 저장하면 자동으로 로드됩니다.

# %%
# === 실행 설정 ===
TARGET_MODEL = "Fine-tuning"  # "RAG" 또는 "Fine-tuning" 선택
DATA_SPLIT = 'train[:10]'     # 처리할 데이터 양 (예: 'train[:10]' -> 10개 샘플)
OUTPUT_FILENAME_BASE = "finance_data_preprocessed"

# === OpenAI API 키 설정 ===
# !!! 중요: 아래 YOUR_OPENAI_API_KEY 부분을 실제 키로 바꾸거나 환경 변수를 사용하세요 !!!
api_key = None

if IS_COLAB:
    try:
        api_key = userdata.get('OPENAI_API_KEY')
        if api_key:
            print("Colab Secrets에서 OpenAI API 키 로드 성공.")
        else:
            print("Colab Secrets에 'OPENAI_API_KEY'가 설정되지 않았습니다.")
    except Exception as e:
        print(f"Colab Secrets 접근 중 오류 발생: {e}")

# Colab이 아니거나 Colab Secrets에 키가 없는 경우, 직접 입력하거나 환경 변수 사용
if not api_key:
    # 1. 직접 입력 (보안에 유의):
    # api_key = "sk-..."
    # 2. 환경 변수 사용 (해당코드에 변수 사용):
    api_key = os.getenv('OPENAI_API_KEY', 'YOUR_OPENAI_API_KEY') # 여기에 키를 직접 넣거나 환경 변수 설정

if not api_key or api_key == 'YOUR_OPENAI_API_KEY':
    print("경고: OpenAI API 키가 설정되지 않았습니다. LLM 초기화가 실패할 수 있습니다.")
    # 여기서 실행을 멈추고 싶다면 raise ValueError("OpenAI API 키가 필요합니다.") 사용 가능
else:
    # LangChain이 환경 변수를 사용하도록 설정 (선택 사항, ChatOpenAI 생성 시 직접 전달도 가능)
    os.environ['OPENAI_API_KEY'] = api_key
    print("OpenAI API 키 설정 완료.")

Colab Secrets에서 OpenAI API 키 로드 성공.
OpenAI API 키 설정 완료.


In [8]:
# 2. LLM 초기화
#
# API 키 설정을 바탕으로 LangChain의 ChatOpenAI 클라이언트를 초기화합니다.
# === LLM 초기화 ===
try:
    llm = ChatOpenAI(
        model_name="gpt-4o",
        temperature=0.0,
        request_timeout=120
        # api_key=api_key # 환경변수 대신 직접 전달하려면 주석 해제
        )
    print(f"LLM 초기화 완료 (gpt-4o)")

except Exception as e:
    print(f"LLM 초기화 오류: {e}")
    print("API 키가 올바르게 설정되었는지, 유효한지 확인하세요.")

LLM 초기화 완료 (gpt-4o)


In [9]:
# 3. 데이터 로딩 함수 정의

# === 데이터 로딩 함수 ===
def load_financial_data(dataset_name: str = "aiqwe/FinShibainu", config_name: str = "qa", split: str = 'train[:1000]') -> pd.DataFrame:
    """Hugging Face에서 금융 데이터셋을 로드하고 필요한 컬럼을 확인합니다."""
    try:
        print(f"데이터셋 로딩 중: '{dataset_name}' ({config_name}, split='{split}')...")
        dataset = load_dataset(dataset_name, config_name, split=split)
        df = dataset.to_pandas()
        print(f"데이터셋 로드 성공. 총 {len(df)}개의 샘플.")

        # 필수 컬럼 확인 및 생성
        required_cols = ['question', 'answer_A', 'answer_B', 'preference_desc']
        for col in required_cols:
            if col not in df.columns:
                print(f"경고: '{col}' 컬럼이 없어 빈 값으로 채웁니다.")
                df[col] = "" # 또는 None, np.nan 등 상황에 맞게

        # 데이터 타입 확인 및 변환
        # 예: df['question'] = df['question'].astype(str)

        return df
    except Exception as e:
        print(f"데이터셋 로딩 중 오류 발생: {e}")
        print(traceback.format_exc())
        return pd.DataFrame() # 오류 시 빈 데이터프레임 반환

In [10]:
# 4. LLM 기반 전처리 함수 정의
# 목표 모델 유형(RAG, Fine-tuning)에 따라 필요한 전처리 함수들을 정의.

# === LLM 기반 전처리 함수 정의 ===

# --- 공통 사용 함수 ---
def get_quality_score(question: str, answer: str, llm_instance: ChatOpenAI) -> Optional[int]:
    """품질 점수(1-5) 평가 (RAG, Fine-tuning 공통 사용 가능)"""
    if not question or not answer: return None
    prompt_template = """
    주어진 질문과 답변을 읽고 명확성, 정확성, 완결성을 고려하여 답변의 전반적인 품질을 1점(매우 나쁨)부터 5점(매우 좋음)까지 평가해주세요.
    다른 설명 없이 **숫자 하나만** 출력해주세요. 예: 4

    [질문]
    {question}
    [답변]
    {answer}
    [품질 점수 (1-5)]
    """
    prompt = PromptTemplate.from_template(prompt_template)
    chain = prompt | llm_instance | StrOutputParser()
    try:
        response = chain.invoke({"question": question, "answer": answer})
        score_str = ''.join(filter(str.isdigit, response))
        if score_str:
            score = int(score_str)
            return score if 1 <= score <= 5 else None
        return None
    except Exception as e:
        # print(f"품질 점수 평가 오류: {e}") # 너무 많은 로그 방지 위해 주석 처리
        return None

# --- RAG 최적화 함수 ---
def rewrite_for_rag_context(answer: str, question: Optional[str], llm_instance: ChatOpenAI) -> Optional[str]:
    """RAG 검색 결과로 사용하기 좋도록 답변을 명확하고 간결하게 재작성합니다."""
    if not answer: return None
    question_context = f"\n\n이 답변은 다음 질문에 대한 것입니다:\n{question}" if question else ""
    prompt_template = """
    당신은 RAG 시스템의 검색 컨텍스트를 최적화하는 전문가입니다. 다음 [원본 답변] 내용을 바탕으로, 검색되었을 때 LLM이 답변을 생성하기 쉽도록 명확하고, 정보 중심적이며, 가능하면 간결하게 내용을 재작성해주세요. 답변 자체만으로도 의미가 어느 정도 전달되도록 해주세요.{question_context}

    [원본 답변]
    {answer}

    [RAG 최적화된 답변]
    """
    prompt = PromptTemplate.from_template(prompt_template)
    chain = prompt | llm_instance | StrOutputParser()
    try:
        response = chain.invoke({"answer": answer, "question_context": question_context})
        return response.strip() if response else None
    except Exception as e:
        # print(f"RAG 답변 재작성 오류: {e}")
        return None

def extract_entities_for_rag(answer: str, llm_instance: ChatOpenAI) -> Optional[str]:
    """답변 텍스트에서 주요 개체명(회사, 상품, 규제 등)이나 핵심 금융 용어를 추출합니다."""
    if not answer: return None
    prompt_template = """
    다음 텍스트에서 중요한 금융 관련 개체명(회사 이름, 금융 상품, 법규 등)이나 핵심 용어를 최대 7개까지 추출하여 쉼표(,)로 구분하여 나열해주세요.
    다른 설명 없이 **추출된 키워드 목록만** 출력해주세요. 예: 삼성전자, 주식, 금리, 예금자보호법

    [입력 텍스트]
    {answer}

    [추출된 개체명/용어]
    """
    prompt = PromptTemplate.from_template(prompt_template)
    chain = prompt | llm_instance | StrOutputParser()
    try:
        response = chain.invoke({"answer": answer})
        return response.strip() if response else None
    except Exception as e:
        # print(f"RAG 개체명 추출 오류: {e}")
        return None

# --- Fine-tuning 최적화 함수 ---
def generate_multiple_question_variations(question: str, num_variations: int, llm_instance: ChatOpenAI) -> Optional[List[str]]:
    """Fine-tuning을 위해 의미는 같지만 표현이 다른 질문 여러 개를 생성합니다."""
    if not question: return None
    prompt_template = """
    당신은 다양한 질문 표현 생성 AI입니다. 다음 원본 질문과 의미는 같지만 다른 표현(구어체, 다른 단어 사용 등)을 사용하여 질문 {num_variations}개를 새로 작성해주세요.
    각 변형 질문은 **별도의 줄**에 작성해주세요. 다른 설명은 절대 포함하지 마세요.

    [원본 질문]
    {question}

    [변형된 질문 목록 ({num_variations}개)]
    """
    prompt = PromptTemplate.from_template(prompt_template)
    chain = prompt | llm_instance | StrOutputParser()
    try:
        response = chain.invoke({"question": question, "num_variations": num_variations})
        variations = [line.strip() for line in response.splitlines() if line.strip()] # 빈 줄 제거
        return variations if variations else None
    except Exception as e:
        # print(f"Fine-tuning 질문 변형 생성 오류: {e}")
        return None

def standardize_answer_style(answer: str, target_style: str, llm_instance: ChatOpenAI) -> Optional[str]:
    """Fine-tuning 데이터의 일관성을 위해 답변을 지정된 스타일로 표준화합니다."""
    if not answer or not target_style: return None
    prompt_template = """
    당신은 금융 답변 스타일 표준화 전문가입니다. 다음 [원본 답변]의 핵심 정보는 유지하면서, [목표 스타일]에 맞게 문체를 변환하여 다시 작성해주세요.

    [원본 답변]
    {answer}

    [목표 스타일]
    {target_style}

    [스타일 표준화된 답변]
    """
    prompt = PromptTemplate.from_template(prompt_template)
    chain = prompt | llm_instance | StrOutputParser()
    try:
        response = chain.invoke({"answer": answer, "target_style": target_style})
        return response.strip() if response else None
    except Exception as e:
        # print(f"Fine-tuning 스타일 표준화 오류: {e}")
        return None

In [11]:
# ## 5. 메인 전처리 로직 함수 정의

# 위에서 정의한 함수들을 사용하여, 목표 모델 유형에 따라 데이터프레임 전체에 전처리를 적용하는 함수.

# === 메인 전처리 로직 ===

def preprocess_data_with_llm_targeted(
    df: pd.DataFrame,
    llm_instance: ChatOpenAI,
    target_model_type: str, # "RAG" 또는 "Fine-tuning"
    sample_size: Optional[int] = None, # 샘플링은 데이터 로드 시 하는 것을 권장
    sleep_time: float = 0.1, # API 호출 간 짧은 대기 시간 (Rate Limit 방지)
    ft_target_style: str = "명확하고 간결하며 중립적인 전문가 톤", # Fine-tuning용 목표 스타일
    ft_num_variations: int = 3 # Fine-tuning용 질문 변형 개수
) -> pd.DataFrame:
    """데이터프레임에 목표 모델 타입에 맞는 LLM 기반 전처리 작업을 적용합니다."""

    if df.empty:
        print("입력 데이터프레임이 비어있어 전처리를 건너<0xEB><0x9B><0x84>니다.")
        return df

    # 샘플링 처리 (주: 이미 load_financial_data에서 split으로 샘플링됨)
    if sample_size is not None and sample_size < len(df):
        print(f"주의: 함수 내에서 추가 샘플링 ({sample_size}개) 진행.")
        df_processed = df.sample(n=sample_size, random_state=42).copy()
    else:
        df_processed = df.copy() # 원본 데이터프레임 변경 방지

    print(f"총 {len(df_processed)}개 행에 대해 '{target_model_type}' 목표 LLM 전처리를 시작합니다...")

    # 각 작업 전에 작은 대기 시간 함수 정의
    def apply_with_delay(series, func, desc):
        results = []
        for item in tqdm(series, desc=desc):
            results.append(func(item))
            time.sleep(sleep_time) # 각 행 처리 후 짧게 대기
        return pd.Series(results, index=series.index)

    # progress_apply 대신 수동 루프와 tqdm 사용 고려 (더 세밀한 제어 가능)
    # 여기서는 progress_apply를 유지하되, 중간 sleep 추가

    if target_model_type == "RAG":
        print("1/3: RAG - 답변 재작성 중...")
        df_processed['rag_answer_context'] = df_processed.progress_apply(
            lambda row: rewrite_for_rag_context(row.get('answer_B') or row.get('answer_A'), row.get('question'), llm_instance), axis=1
        )
        print("약간의 대기...")
        time.sleep(sleep_time * 5) # 작업 그룹 간 조금 더 긴 대기

        print("2/3: RAG - 개체명/키워드 추출 중...")
        df_processed['rag_entities'] = df_processed.progress_apply(
            lambda row: extract_entities_for_rag(row.get('rag_answer_context') or row.get('answer_B') or row.get('answer_A'), llm_instance), axis=1
        )
        print("약간의 대기...")
        time.sleep(sleep_time * 5)

        print("3/3: RAG - 품질 점수 평가 중...")
        df_processed['llm_quality_score'] = df_processed.progress_apply(
            lambda row: get_quality_score(row['question'], row.get('rag_answer_context') or row.get('answer_B') or row.get('answer_A'), llm_instance), axis=1
        )

    elif target_model_type == "Fine-tuning":
        print("1/3: Fine-tuning - 품질 점수 평가 중...")
        df_processed['llm_quality_score'] = df_processed.progress_apply(
            lambda row: get_quality_score(row['question'], row.get('answer_B') or row.get('answer_A'), llm_instance), axis=1
        )
        print("약간의 대기...")
        time.sleep(sleep_time * 5)

        print("2/3: Fine-tuning - 질문 변형 생성 중...")
        df_processed['ft_question_variations'] = df_processed.progress_apply(
            lambda row: generate_multiple_question_variations(row['question'], ft_num_variations, llm_instance), axis=1
        )
        print("약간의 대기...")
        time.sleep(sleep_time * 5)

        print("3/3: Fine-tuning - 답변 스타일 표준화 중...")
        df_processed['ft_standardized_answer'] = df_processed.progress_apply(
            lambda row: standardize_answer_style(row.get('answer_B') or row.get('answer_A'), ft_target_style, llm_instance), axis=1
        )

    else:
        print(f"경고: 알 수 없는 target_model_type ('{target_model_type}'). 전처리를 수행하지 않습니다.")
        return df_processed # 원본 반환

    print("LLM 전처리 완료.")
    return df_processed

In [12]:
# 6. 실행: 데이터 로드 및 전처리

# 위에서 정의한 함수와 설정을 사용하여 실제 데이터 로딩 및 전처리를 수행합니다.

# === 실행 메인 로직 ===

print("LLM 기반 타겟 모델 최적화 전처리 시작...")

# 1. 원본 데이터 로드 (설정된 DATA_SPLIT 사용)
# LLM 인스턴스(llm)가 성공적으로 생성되었는지 확인 후 진행
if 'llm' in locals() and isinstance(llm, ChatOpenAI):
    raw_df = load_financial_data(split=DATA_SPLIT)

    # 2. 데이터 로드 성공 시 전처리 실행
    if not raw_df.empty:
        print(f"\n'{TARGET_MODEL}' 목표 전처리를 진행합니다.")
        # sample_size=None으로 설정하여 로드 시 적용된 split 사용
        processed_df = preprocess_data_with_llm_targeted(
            raw_df,
            llm, # 초기화된 LLM 인스턴스 전달
            target_model_type=TARGET_MODEL,
            sample_size=None,
            sleep_time=0.2 # API 호출 간격 조정 가능
            # ft_target_style, ft_num_variations 는 함수 기본값 사용
        )

        # 3. 결과 확인 (상위 5개 행)
        print(f"\n--- '{TARGET_MODEL}' 전처리 결과 확인 (상위 5개 행) ---")
        display_cols = ['question'] # 기본 표시 컬럼
        if TARGET_MODEL == "RAG":
            display_cols.extend(['answer_B', 'rag_answer_context', 'rag_entities', 'llm_quality_score'])
        elif TARGET_MODEL == "Fine-tuning":
             display_cols.extend(['answer_B', 'llm_quality_score', 'ft_question_variations', 'ft_standardized_answer'])

        # DataFrame에 실제로 존재하는 컬럼만 필터링
        display_cols = [col for col in display_cols if col in processed_df.columns]

        # Jupyter Notebook에서는 print 대신 바로 DataFrame을 출력하는 것이 보기 좋을 수 있음
        display(processed_df[display_cols].head()) # display() 사용 권장

    else:
        print("\n데이터 로드에 실패하여 전처리를 진행할 수 없습니다.")

else:
    print("\nLLM이 초기화되지 않아 전처리를 진행할 수 없습니다. API 키 설정을 확인하세요.")

LLM 기반 타겟 모델 최적화 전처리 시작...
데이터셋 로딩 중: 'aiqwe/FinShibainu' (qa, split='train[:10]')...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.05k [00:00<?, ?B/s]

train-00000-of-00006.parquet:   0%|          | 0.00/13.4M [00:00<?, ?B/s]

train-00001-of-00006.parquet:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

약간의 대기...
3/3: Fine-tuning - 답변 스타일 표준화 중...


  0%|          | 0/10 [00:00<?, ?it/s]

LLM 전처리 완료.

--- 'Fine-tuning' 전처리 결과 확인 (상위 5개 행) ---


,question,answer_B,llm_quality_score,ft_question_variations,ft_standardized_answer
0,"가계부문의 순저축률을 측정하는 공식을 적고, 각 항목이 의미하는 바를 설명하시오.","가계부문의 순저축률은 가계가 얼마나 저축하고 있는지를 나타내는 중요한 경제 지표로,...",5,"[가계의 순저축률을 계산하는 방법을 쓰고, 그 안에 포함된 항목들이 무엇을 뜻하는지...",가계부문의 순저축률은 가계의 저축 수준을 나타내는 중요한 경제 지표입니다. 이는 다...
1,"로보어드바이저가 가지고 있는 장점은 무엇이며, 특히 소액 자산가에게 어떤 이점을 제...",로보어드바이저는 인공지능과 알고리즘을 활용하여 투자 포트폴리오를 관리하는 자동화된 ...,5,"[로보어드바이저의 강점은 뭐고, 특히 돈이 많지 않은 사람들에게 어떤 도움이 되는지...",로보어드바이저는 인공지능과 알고리즘을 활용하여 투자 포트폴리오를 자동으로 관리하는 ...
2,로보어드바이저의 의미와 그 서비스가 어떻게 제공되는지 설명하세요.,로보어드바이저(Robo-Advisor)는 인공지능(AI) 및 알고리즘을 기반으로 투...,5,"[로보어드바이저가 뭔지랑 그 서비스가 어떤 식으로 이루어지는지 알려줄래?, 로보어드...",로보어드바이저는 인공지능과 알고리즘을 활용하여 투자 자산 관리 서비스를 제공하는 온...
3,"경상수지와 재정수지의 차이점에 대해 설명하고, 각각이 경제에 미치는 영향을 서술하시오.",경상수지(Current Account)와 재정수지(Fiscal Balance or ...,5,"[경상수지랑 재정수지가 어떻게 다른지 설명해주고, 이 둘이 경제에 어떤 영향을 주는...",경상수지와 재정수지는 국가 경제의 건강성을 평가하는 중요한 지표입니다. 그러나 이들...
4,표면이율이 낮은 채권이 매매수익률 대비 높은 세후수익률을 얻는 이유는 무엇인가?,채권 투자에서 표면이율(Coupon Rate)과 매매수익률(Yield to Matu...,5,"[표면이율이 낮은 채권이 매매수익률보다 세후수익률이 더 높은 이유가 뭘까?, 왜 표...","채권 투자에서 표면이율, 매매수익률, 세후수익률 간의 관계는 여러 요인에 의해 복잡..."


In [13]:
# 7. 결과 저장

# 전처리된 데이터프레임을 CSV 파일로 저장합니다.

# === 결과 저장 ===
if 'processed_df' in locals() and not processed_df.empty:
    try:
        output_filename = f"{OUTPUT_FILENAME_BASE}_{TARGET_MODEL.lower()}.csv"
        processed_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"\n전처리된 데이터가 '{output_filename}'으로 저장되었습니다.")
    except Exception as e:
        print(f"\n결과 저장 중 오류 발생: {e}")
        print(traceback.format_exc())
elif 'raw_df' in locals() and raw_df.empty:
     print("\n원본 데이터 로드 실패로 저장할 결과가 없습니다.")
elif 'llm' not in locals() or not isinstance(llm, ChatOpenAI):
    print("\nLLM 초기화 실패로 저장할 결과가 없습니다.")
else:
    print("\n처리된 데이터프레임('processed_df')이 없어 결과를 저장할 수 없습니다.")


print("\n노트북 셀 실행 완료.")


전처리된 데이터가 'finance_data_preprocessed_fine-tuning.csv'으로 저장되었습니다.

노트북 셀 실행 완료.
